In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib

from IPython import display

In [10]:
# d1 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/client'
# d2 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/counselor'
# # iterate over files in
# # that directory

# data_in = []
# data_out = []
# for filename in sorted(os.listdir(d1)):
#     fn = os.path.join(d1, filename)
#     if os.path.isfile(fn):
#         with open(fn) as f:
#             lines = f.readlines()
#             aline = []
#             for x in lines:
#                 aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
#             data_in.append(aline)
#         f.close()


# for filename in sorted(os.listdir(d2)):
#     fn = os.path.join(d2, filename)
#     # checking if it is a file
#     if os.path.isfile(fn):
#         with open(fn) as f:
#             lines = f.readlines()
#             aline = []
#             for x in lines:
#                 aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
#             data_out.append(aline)
#         f.close()
        

In [11]:
# data_in = data_in[:len(data_out)]
# # print(tf.constant(data_in).shape)
# # print(tf.constant(data_out).shape)

# data = []
# sequence_len = 16
# ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
# ds = ds.batch(sequence_len)
# ds = ds.shuffle(ds.cardinality().numpy())
# for inn, outt in ds:
#     print(inn.shape, outt.shape)

In [43]:
class GU(Model):

    def __init__(self, in_dim, num_channels=2, emb_dim=512, gate_filters=32, num_resolutions=2, attn_res_idx=0):
        super().__init__()
        
        self.in_dim = in_dim
        self.num_channels = num_channels
        self.num_resolutions = num_resolutions
        self.attn_res_idx = attn_res_idx
        self.emb_dim = emb_dim
        self.gate_filters = gate_filters
#         self.model = self.build_graph()
        
#     def timestep_embedding(self, ts, embedding_dim):
#         assert len(ts.shape)==1
        
#         half_dim = embedding_dim // 2
#         emb = -(tf.math.log(10000.0)/(half_dim-1))
        
#         idxs = tf.range(half_dim, dtype=tf.float32)
#         emb = tf.math.exp(idxs*emb)
        
#         ts = tf.cast(ts, dtype=tf.float32)
        
#         emb = ts[:,None]* emb[None,:]
        
#         pe = tf.concat([tf.sin(emb), tf.cos(emb)], axis=1)
        
#         return pe
    
    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
        
    def ResBlK(self, x, temb=None):
        h = self.nonlinearity(self.normalize(x))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        if not temb == None:
            temb = self.nonlinearity(temb)
            h +=  tf.reshape(layers.Dense(h.shape[-1])(temb), (-1, 1, 1,h.shape[-1]))

        h = self.nonlinearity(self.normalize(h))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return x+h
        
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)

    def upsample(self, x, filters, factor=2):
        return layers.Conv2DTranspose(filters, (3, 3),
                                       strides=(factor, factor), padding='same')(x)
        
    def NonlocalGaussian(self, x):
        h = self.normalize(x)
        theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = tf.transpose(phi, (0, 3, 2, 1))

        theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

        phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
        g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
        f = tf.matmul(theta, phi)
        
        f = layers.Softmax()(f)
        
        y = tf.matmul(f, g)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    
    def design(self, img_input):
        
        hs = [layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(img_input)]
        
        for i in range(self.num_resolutions):
            h = self.ResBlK(hs[-1])
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)
                
            if i != self.num_resolutions - 1:
                h = self.downsample(h, h.shape[-1]*2)
                
            hs.append(h)

        h = hs[-1]
        h = self.ResBlK(h)
        h = self.NonlocalGaussian(h)
        h = self.ResBlK(h)
                
        for i in reversed(range(self.num_resolutions)):
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)   
            h = self.ResBlK(h)
            h = h+hs[i]
                
            if i != 0:
                h = self.upsample(h, h.shape[-1]//2)
                
        h = self.nonlinearity(self.normalize(h))
        
        h = layers.Conv2D(self.num_channels, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return h
    
#     def call(self, x):
#         return self.model([x])
    
    def build_model(self):
        x = layers.Input(shape=(self.in_dim[0],
                                        self.in_dim[1], self.num_channels))
        
        return Model(inputs=[x], outputs=self.design(x))
        

In [44]:
unet1=GU((16, 68))

In [45]:
model1 = unet1.build_model()

In [52]:
# unet1(tf.zeros(shape=(16,68,2)))

In [53]:
# tf.keras.utils.plot_model(model1,to_file="./imgs/model.png", show_shapes=True, show_layer_activations=True)

In [23]:
EPSILON = 1e-16

class model:
    def __init__(self, dataset_path, data_shape, num_channels, dum=5, train_split=0.8,
                 batch_size=64, lr=3e-4):
        self.s = data_shape
        self.nc = num_channels
        self.bs = batch_size
        self.dum = dum
        
        if not dataset_path==None:
            cl_path = dataset_path + "/client"
            co_path = dataset_path + "/counselor"

            self.ds = self.make_dataset(cl_path, co_path)
            
            ds_size = self.ds.cardinality().numpy()
            train_size = int(train_split * ds_size)
            test_size = int((1.0-train_split) * ds_size)

            self.train_ds = self.ds.take(train_size)    
            self.test_ds = self.ds.skip(train_size)
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        
        self.g = U_Net(self.s, self.nc)
        self.g_opt = tf.keras.optimizers.Adam(lr)

    def make_dataset(self, d1, d2):
        data_in = []
        data_out = []
        for filename in sorted(os.listdir(d1)):
            fn = os.path.join(d1, filename)
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_in.append(aline)
                f.close()


        for filename in sorted(os.listdir(d2)):
            fn = os.path.join(d2, filename)
            # checking if it is a file
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_out.append(aline)
                f.close()

        data_in = data_in[:len(data_out)]
        data = []
        sequence_len = self.s[0]
        ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
        ds = ds.batch(sequence_len)
        ds = ds.shuffle(ds.cardinality().numpy())
        
        return ds
    
    def loss_func(self, pred_co, gt_co):
        return tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.add(pred_co, -tf.cast(gt_co, tf.float32))**2, axis = [1,2,3])+EPSILON))
        
    @tf.function
    def update(self, cl, co):
        with tf.GradientTape() as g_tape:
            pred_co = self.g(cl)
            g_loss = self.loss_func(pred_co, co)
            
        grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
        self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        
        return g_loss
        
    def train(self, epochs=50):
        for epo in range(epochs):
            g_losses = []
            
            batch_inn = []
            batch_outt = []

            for inn, outt in self.train_ds:
                if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                    if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                        tf_batch_inn = tf.stack(batch_inn)
                        tf_batch_outt = tf.stack(batch_outt)
                        g_losses.append(self.update(tf_batch_inn, tf_batch_outt))
                        batch_inn = []
                        batch_outt = []
                    else:
                        batch_inn.append(inn)
                        batch_outt.append(outt)


            print("Epoch {:04d}".format(epo), "Generator Avg. Loss: ", np.mean(g_losses), flush=True)
    def test(self):
        errors = []
        batch_inn = []
        batch_outt = []
        for inn, outt in self.test_ds:
            if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                    tf_batch_inn = tf.stack(batch_inn)
                    tf_batch_outt = tf.stack(batch_outt)
                    pred_co = self.g(tf_batch_inn)
                    errors.append(self.loss_func(pred_co, tf_batch_outt))
                    batch_inn = []
                    batch_outt = []
                else:
                    batch_inn.append(inn)
                    batch_outt.append(outt)
        print("Test Set Avg. Loss: ", np.mean(errors), flush=True)
        

In [24]:
ds_path = "/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2"

In [25]:
m1 = model(ds_path, (8,68), 2)

In [1]:
# m1.train(10000)

In [2]:
# m1.test()

In [3]:
# batch_size=32
# batch_inn = []
# batch_outt = []


# for inn, outt in m1.ds:
#     if inn.shape == (8,68,2) and outt.shape == (8,68,2):
#         if (len(batch_inn))%batch_size ==0:
#             tf_batch_inn = tf.stack(batch_inn)
#             tf_batch_outt = tf.stack(batch_outt)
#             if not tf_batch_inn.shape[0] == 0:
#                 pred_out = m1.g(tf_batch_inn)
#                 print(pred_out.shape)
#             batch_inn = []
#             batch_outt = []
#             batch_inn.append(inn)
#             batch_outt.append(outt)
#         else:
#             batch_inn.append(inn)
#             batch_outt.append(outt)
#         print(batch_outt)
